# Import required libraries

In [13]:
import dotenv
import openai
import os
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch

# Configure OpenAI settings

In [21]:
dotenv.load_dotenv()
openai.api_type = "azure"
openai.api_version = "2023-05-15" 
openai.api_base = os.getenv("AZURE_OPENAI_API_BASE")
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")
model = "text-embedding-ada-002"

# Configure vector store settings

In [23]:
vector_store_address = os.getenv("AZURE_COGNITIVE_SEARCH_URL")
vector_store_password = os.getenv("AZURE_COGNITIVE_SEARCH_KEY")

# Create embeddings and vector store instances

In [25]:
embeddings = OpenAIEmbeddings(deployment=model, chunk_size=1)
index_name = "acs-demo"
vector_store = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

# Insert text and embeddings into vector store

In [28]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

loader = TextLoader("state_of_the_union.txt", encoding="utf-8")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

vector_store.add_documents(documents=docs)

['ZDAxMzM3MjctYTJhZi00YzBkLTgxNzctYzA0MDBkMTdlOTg2',
 'MjdmOWU0ZjktMTkzYi00ZjZkLWJlMWEtNDI0OTAxZTMyNzcw',
 'MjQ2ZmJlZjAtMTg4NS00YTNmLWIxOTEtM2EzZDlkY2ViYjQ0',
 'YjA1ZGNlNmQtNjEzYi00OWM5LTlmYmQtOWU4YjYzNzNmNmI4',
 'NTkxOWJmYzktOWVhZi00OTdmLTlhMTYtN2FjYTRlYmM4NWYx',
 'N2U1NzczOTQtNTIzNy00ODNhLWI4YWYtNjE2NGJlY2JjYjcy',
 'YTQxOGUwNDUtNDVmNS00MTg4LWIxODctODUwZmIyOTc1MWEy',
 'ZmI3YTM1MDItMzkzZS00NmFlLWE4NWUtNjUwYjBlM2IwNWNi',
 'NzYyMTM4ZjgtMDNlMi00OWIzLTgyZDctNTBjMTMyYzJmYTNi',
 'MDlkZTFmNDYtMTRlYS00YjYzLTgyZTgtMDBhZmE1NjhmZWZi',
 'ZDE0ZGMzOWYtYzA1Mi00Njc0LWEwM2MtNGZlNTdhMmE5OTAw',
 'YjNhYTkxMzctYjMwMi00NWIzLTkxZmQtYWUyYWQ2N2RhYjY1',
 'MzlkMjQ4NmQtOTk1NS00NzBjLWExN2UtYmQ0MTFlN2U3M2U0',
 'ZDc4NTlhODEtNjljNS00ZjdhLTlmNzQtNDQxYTM5ZTg1Mzk2',
 'NjhhM2FlZmYtNDk4Yi00N2RiLTkyMWUtNzlkODRmYzUwMjA4',
 'YjViM2UyODUtMWRhZC00NjNkLTlhMGMtN2NhMjZiYmZmZTlk',
 'NmQ3MDRhMzMtNmEwMy00MzhiLTllOGQtODQ5NmVjZGM2YmJh',
 'NjY4OGM3MzEtZjdjZC00ZTRhLWIxYWYtNmVkMTA5ZDY2OTcy',
 'MzI2ZjA0OTMtMWY2MS00MGJjLTg0N2YtZDNmZGE0NDgy

# Perform a vector similarity search

In [29]:
docs = vector_store.similarity_search(
    query="What did the president say about Ketanji Brown Jackson",
    k=3,
    search_type="similarity",
)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
